# Personalized emails with Amazon Personalize and Generative AI - Part 2<a class="anchor" id="top"></a>


## Outline

1. [Introduction](#intro)
1. [Get Personalized recommendations from Amazon Personalize](#getRecs)
1. [Get the user's favorite movie genre](#getGenre)
1. [Add User demographic information](#getDemographic)
1. [Using Amazon Bedrock](#Bedrock)
1. [Generate personalized Marketing Communication](#Emails)
1. [Wrap up](#wrapup)



## Introduction <a class="anchor" id="outline"></a>

In the previous notebook: [`Media_03_Training.ipynb`](Media_03_Training.ipynb) you trained and evaluated an Amazon Personalize top-pics-for-you recommender to generate personalize recommendations for each user.

In this Notebook we will get recommendations using the Amazon Personalzie recommender we trainined in the previous notebook including the item metadata and generate Personalized marketing communication for different users experimenting with different prompts and user demographics.


Connect to Amazon Personalize via [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html), the AWS SDK for Python and check the resources have been created.

Similar to the previous notebook, start by importing the relevant packages, and set up a connection to Amazon Personalize using the SDK.

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import botocore
from botocore.exceptions import ClientError
import pandas as pd
import re

In [18]:
# retrive the saved variables from the previous notebook
%store -r

## Get Personalized recommendations from Amazon Personalize<a class="anchor" id="getRecs"></a>
[Back to top](#top)

Now that we have trained the [Top picks for you](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#top-picks-use-case) recommender we can get recommendations for our users. 

For more details and ways to use Amazon Personalize to get recommendations, please see the [Amazon Personalize Getting Recommendations](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html).

In [3]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

Select a random user to see their recommendations.

In [7]:
users = random.sample(range(1, 600), 3)
user_id = users[0]

Get 15 recommendations from the 'Top pics for you' recommender we trained.

In [8]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = workshop_recommender_top_picks_arn,
    userId = str(user_id),
    numResults = 15,
    metadataColumns = {
        "ITEMS": ['TITLE', 'PLOT', 'GENRES']
    }
)

print (get_recommendations_response['itemList'])

[{'itemId': 'tt0107822', 'metadata': {'genres': 'Drama|Music|Romance', 'plot': 'In the mid-19th century, a mute woman is sent to New Zealand along with her young daughter and prized piano for an arranged marriage to a wealthy landowner, but is soon lusted after by a local worker on the plantation.', 'title': 'The Piano'}}, {'itemId': 'tt0108399', 'metadata': {'genres': 'Crime|Drama|Romance|Thriller', 'plot': 'In Detroit, a lonely pop culture geek marries a call girl, steals cocaine from her pimp, and tries to sell it in Hollywood. Meanwhile, the owners of the cocaine, the Mob, track them down in an attempt to reclaim it.', 'title': 'True Romance'}}, {'itemId': 'tt0110932', 'metadata': {'genres': 'Biography|Drama|History', 'plot': "A young lawyer, Richard Goodwin, investigates a potentially fixed game show. Charles Van Doren, a big time show winner, is under Goodwin's investigation.", 'title': 'Quiz Show'}}, {'itemId': 'tt0105151', 'metadata': {'genres': 'Comedy|Crime|Drama|Thriller', '

It works, we can get recommendations!

Let's create a method to get recommended movies and their metadata for each user.

In [9]:
def getRecommendedMoviesForUserId(
    user_id, 
    workshop_recommender_top_picks_arn, 
    item_data, 
    number_of_movies_to_recommend = 5):
    # For a user_id, gets the top n (number_of_movies_to_recommend) movies using Amazon Personalize 
    # and gets the additional metadata for each movie (item_id) from the item_data.
    # returns a list of movie dictionaries (movie_list) with the relevant data.

    # get recommended movies
    get_recommendations_response = personalize_runtime.get_recommendations(
        recommenderArn = workshop_recommender_top_picks_arn,
        userId = str(user_id),
        numResults = number_of_movies_to_recommend,
        metadataColumns = {
            "ITEMS": ['TITLE', 'PLOT', 'GENRES']
        }
    )

    # create a list of movies with title, geners, and plot.   
    movie_list = []
    
    for recommended_movie in get_recommendations_response['itemList']:      
        movie_list.append(
            {
                'title' : recommended_movie['metadata']['title'],
                'genres' : recommended_movie['metadata']['genres'].replace('|', ' and '),
                'plot' : recommended_movie['metadata']['plot']
            }
        )
    return movie_list
    

We select a random user_id.

In [11]:
users = random.sample(range(1, 600), 3)
user_id = users[0]

We can select how many movies we want to recommend. In this case 3.

In [12]:
number_of_movies_to_recommend = 3 

Let's get the recommended movies for the user. First though lets load the item data

In [14]:
# read movielens data
item_data = pd.read_csv(data_dir + '/imdb/items.csv', sep=',', dtype={'PROMOTION': "string"})
item_data.head(5)

,ITEM_ID,TITLE,YEAR,IMDB_RATING,IMDB_NUMBEROFVOTES,PLOT,US_MATURITY_RATING_STRING,US_MATURITY_RATING,GENRES,CREATION_TIMESTAMP,PROMOTION
0,tt0906325,Bagboy,2007,4,741,A teenager enters the competitive world of gro...,PG-13,13,Comedy,1167609600,false
1,tt0906665,Sukiyaki Western Django,2007,6,14582,A nameless gunfighter arrives in a town ripped...,R,17,Action|Western,1167609600,false
2,tt0907657,Once,2007,8,109787,A modern-day musical about a busker and an imm...,R,17,Drama|Music|Romance,1167609600,false
3,tt0910905,In the Electric Mist,2009,6,16592,A detective in post-Katrina New Orleans has a ...,R,17,Crime|Drama|Mystery|Thriller,1230768000,false
4,tt0910554,Frequently Asked Questions About Time Travel,2009,7,32051,"While drinking at their local pub, three socia...",PG-13,13,Comedy|Sci-Fi,1230768000,false


In [15]:
movie_list = getRecommendedMoviesForUserId(user_id, workshop_recommender_top_picks_arn, item_data, number_of_movies_to_recommend)

# print each movie in the array
for movie in movie_list:
    print ('Title: '+movie['title'])
    print ('Genres: '+movie['genres'])
    print ('Plot: '+movie['plot'])
    print ()

Title: Big Daddy
Genres: Comedy and Drama
Plot: A lazy law school grad adopts a kid to impress his girlfriend, but everything doesn't go as planned and he becomes the unlikely foster father.

Title: Pretty Woman
Genres: Comedy and Romance
Plot: A man in a legal but hurtful business needs an escort for some social events, and hires a beautiful prostitute he meets... only to fall in love.

Title: 10 Things I Hate About You
Genres: Comedy and Drama and Romance
Plot: A pretty, popular teenager can't go out on a date until her ill-tempered older sister does.



## Get the user's favorite movie genre<a class="anchor" id="getGenre"></a>
[Back to top](#top)

In order to provide a better personalized marketing communication, in this section we calculate the user's favorite movie genre based on the genres of all the movies they have interacted with in the past.

In [16]:
def getUserFavouriteGenres(user_id, interactions_df, movie_data):
    # For a user_id, gets the users favourite genre by looking at the user's interactions 
    # with each movie in the past and counting the genres to find the most comon genre. 

    # Get all movies the user has watched     
    movies_df = interactions_df[interactions_df['USER_ID'] == user_id]

    genres = {}

    for movie_id in movies_df['ITEM_ID']:

        movie_genres = movie_data[movie_data['ITEM_ID']==movie_id]['GENRES']
        
        if not len(movie_genres.tolist())==0:
            for movie_genre in movie_genres.tolist()[0].split('|'):
                if movie_genre in genres:
                    genres[movie_genre] +=1
                else:
                    genres[movie_genre] = 1

    genres_df = pd.DataFrame(list(genres.items()), columns =['GENRE', 'COUNT'])
    
    # Sort by most common
    genres_df.sort_values(by=['COUNT'], inplace=True, ascending = False)
    
    # Return the most common (favourite) genre       
    return genres_df.iloc[[0]]['GENRE'].values[0]
    

In [19]:
user_favorite_genre = getUserFavouriteGenres(user_id, interactions_df, item_data)
user_favorite_genre

'Comedy'

We get user preferred genres from for this user by counting the number of interactions they have with each genre in the past.

## Using Amazon Bedrock<a class="anchor" id="Bedrock"></a>
[Back to top](#top)

[Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-bedrock.html) is a fully managed service that makes base models from Amazon and third-party model providers accessible through an API.

<div class="alert alert-block alert-warning">
<b>Note:</b> Amazon Bedrock users need to request access to models before they are available for use. If you want to add additional models for text, chat, and image generation, you need to request access to models in Amazon Bedrock. To request access to additional models, select the Model access link in the left side navigation panel in the Amazon Bedrock console. For more information see: https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html
</div>


### Connect to Amazon Bedrock
Connect to Amazon Bedrock via [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html), the AWS SDK for Python.

in this example we will be using [Anthropic Claude](https://aws.amazon.com/bedrock/claude/) on Amazon Bedrock.

In [20]:
bedrock = boto3.client('bedrock-runtime') 

In [21]:
# Model parameters
# The LLM we will be using
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
    

# The maximum number of tokens to use in the generated response
max_tokens_to_sample = 1000

## Add User demographic information<a class="anchor" id="getDemographic"></a>
[Back to top](#top)

We'll generate emails by assuming two different demographics for the users.

<div class="alert alert-block alert-warning">
<b>Note:</b> Please note that the used version ml-latest-small dataset from the Movielens Dataset (https://grouplens.org/datasets/movielens/) does not contain demographic data and therefore we are trying out multiple options. In a real world scenario, you may know the demographics of your audience.
</div>

In [22]:
# Sample user demographics
user_demographic_1 = f'The user is a 50 year old adult called Otto.'
user_demographic_3 = f'The user is a young adult called Jane.'


## Generate Personalized Marketing Emails<a class="anchor" id="Emails"></a>
[Back to top](#top)


### Generating a marketing email using a simple prompt
Let's generate a simple marketing email that just uses the recommended movies.

In [23]:
def generate_personalize_simple_prompt(movie_list, model_id, max_tokens_to_sample = 50):

    prompt_template = f'''\n\nHuman: Write a marketing email advertising several movies available in a video-on-demand streaming platform next week, given the movie and user information below. The movies to recommend and their information is contained in the <movie> tag. Put the email between <email> tags.

    <movie>
    {movie_list}
    </movie>

    Assistant: Email body:
    <email>.

    Assistant:
    '''
    
    if 'claude' in model_id:
        prompt_input = json.dumps({"prompt": prompt_template, "max_tokens_to_sample": max_tokens_to_sample })
    
    return prompt_input

In [24]:
print ('User\'s recommended movies:')

# print each movie in the array
for movie in movie_list:
    print ('Title: '+movie['title'])
    print ('Genres: '+movie['genres'])
    print ('Plot: '+movie['plot'])
    print ()

User's recommended movies:
Title: Big Daddy
Genres: Comedy and Drama
Plot: A lazy law school grad adopts a kid to impress his girlfriend, but everything doesn't go as planned and he becomes the unlikely foster father.

Title: Pretty Woman
Genres: Comedy and Romance
Plot: A man in a legal but hurtful business needs an escort for some social events, and hires a beautiful prostitute he meets... only to fall in love.

Title: 10 Things I Hate About You
Genres: Comedy and Drama and Romance
Plot: A pretty, popular teenager can't go out on a date until her ill-tempered older sister does.



In [25]:
# Create prompt input
prompt_input_json = generate_personalize_simple_prompt( movie_list, model_id, max_tokens_to_sample )
prompt_input_json

'{"prompt": "\\n\\nHuman: Write a marketing email advertising several movies available in a video-on-demand streaming platform next week, given the movie and user information below. The movies to recommend and their information is contained in the <movie> tag. Put the email between <email> tags.\\n\\n    <movie>\\n    [{\'title\': \'Big Daddy\', \'genres\': \'Comedy and Drama\', \'plot\': \\"A lazy law school grad adopts a kid to impress his girlfriend, but everything doesn\'t go as planned and he becomes the unlikely foster father.\\"}, {\'title\': \'Pretty Woman\', \'genres\': \'Comedy and Romance\', \'plot\': \'A man in a legal but hurtful business needs an escort for some social events, and hires a beautiful prostitute he meets... only to fall in love.\'}, {\'title\': \'10 Things I Hate About You\', \'genres\': \'Comedy and Drama and Romance\', \'plot\': \\"A pretty, popular teenager can\'t go out on a date until her ill-tempered older sister does.\\"}]\\n    </movie>\\n\\n    Assi

Let's invoke the model.

In [26]:
def getPersonalizedEmail(bedrock_client, model_id, max_tokens_to_sample, prompt ):
    
    personalized_email = "ERROR"
    
    body = json.dumps({
      "max_tokens": max_tokens_to_sample,
      "messages": [{"role": "user", "content": prompt}],
      "anthropic_version": "bedrock-2023-05-31"
    })
    
    response = bedrock.invoke_model(body=body, modelId=model_id)
    
    response_body = json.loads(response.get('body').read())
        
    # Clean Gen AI response
    personalized_email = re.sub(r'<[^>]*>', '', response_body['content'][0]['text'])

    
    return personalized_email

Next we need to invoke the model. 

<div class="alert alert-block alert-warning">
<b>Note:</b> Amazon Bedrock users need to request access to models before they are available for use. If you get an `Access Denied` error, make sure you have requested access to this model. To request access to additional models, select the Model access link in the left side navigation panel in the Amazon Bedrock console. For more information see: https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html
</div>


In [27]:
# Invoke model
response = getPersonalizedEmail(bedrock,model_id, max_tokens_to_sample, prompt_input_json)
print (response)


Subject: Lights, Camera, Streaming! This Week's Must-See Movies

Dear [Customer's Name],

Calling all movie buffs! We've got an exciting lineup of films hitting our video-on-demand platform next week, and we can't wait to share them with you. From heartwarming comedies to romantic tales, there's something for everyone to enjoy. Here are our top picks:

1. "Big Daddy" (Comedy/Drama): Get ready to laugh and feel all the feels with this hilarious yet touching story. Adam Sandler stars as a lazy law school grad who adopts a kid to impress his girlfriend, but ends up becoming an unlikely foster father. It's a rollercoaster of emotions you won't want to miss!

2. "Pretty Woman" (Comedy/Romance): Julia Roberts and Richard Gere light up the screen in this timeless classic. A businessman hires a beautiful prostitute as his escort for social events, but their professional arrangement blossoms into an unexpected love story. Prepare to swoon!

3. "10 Things I Hate About You" (Comedy/Drama/Romance

### Generating a marketing email using a more advanced prompt
In this example we are adding the favorite genre for the user as well as exploring two different possible demographics in the input.

In [28]:
def generate_personalize_advanced_prompt(user_demographic, favorite_genre, movie_list, model_id, max_tokens_to_sample = 50):

    prompt_template = f'''\n\nHuman: You are a skilled publicist. Write a high-converting marketing email advertising several movies available in a video-on-demand streaming platform next week, 
    given the movie and user information below. Your email will leverage the power of storytelling and persuasive language. 
    You want the email to impress the user, so make it appealing to them based on the information contained in the <user> tags, 
    and take into account the user's favorite genre in the <genre> tags. 
    The movies to recommend and their information is contained in the <movie> tag. 
    All movies in the <movie> tag must be recommended. Give a summary of the movies and why the human should watch them. 
    Put the email between <email> tags.

    <user>
    {user_demographic}
    </user>

    <genre>
    {favorite_genre}
    </genre>

    <movie>
    {movie_list}
    </movie>

    Assistant:
    '''
    
    if 'claude' in model_id:
        prompt_input = json.dumps({"prompt": prompt_template, "max_tokens_to_sample": max_tokens_to_sample })
    
    return prompt_input


### Generating a marketing email for a 50 year old user

In [29]:
print ('User\'s demographic')
user_demographic = user_demographic_1
user_demographic

User's demographic


'The user is a 50 year old adult called Otto.'

In [30]:
print ('User\'s favorite Genre')
user_favorite_genre

User's favorite Genre


'Comedy'

In [31]:
print ('User\'s recommended movies:')

# print each movie in the array
for movie in movie_list:
    print ('Title: '+movie['title'])
    print ('Genres: '+movie['genres'])
    print ('Plot: '+movie['plot'])
    print ()

User's recommended movies:
Title: Big Daddy
Genres: Comedy and Drama
Plot: A lazy law school grad adopts a kid to impress his girlfriend, but everything doesn't go as planned and he becomes the unlikely foster father.

Title: Pretty Woman
Genres: Comedy and Romance
Plot: A man in a legal but hurtful business needs an escort for some social events, and hires a beautiful prostitute he meets... only to fall in love.

Title: 10 Things I Hate About You
Genres: Comedy and Drama and Romance
Plot: A pretty, popular teenager can't go out on a date until her ill-tempered older sister does.



In [32]:
# Create prompt input
prompt_input_json = generate_personalize_advanced_prompt(user_demographic, user_favorite_genre, movie_list, model_id, max_tokens_to_sample )
prompt_input_json

'{"prompt": "\\n\\nHuman: You are a skilled publicist. Write a high-converting marketing email advertising several movies available in a video-on-demand streaming platform next week, \\n    given the movie and user information below. Your email will leverage the power of storytelling and persuasive language. \\n    You want the email to impress the user, so make it appealing to them based on the information contained in the <user> tags, \\n    and take into account the user\'s favorite genre in the <genre> tags. \\n    The movies to recommend and their information is contained in the <movie> tag. \\n    All movies in the <movie> tag must be recommended. Give a summary of the movies and why the human should watch them. \\n    Put the email between <email> tags.\\n\\n    <user>\\n    The user is a 50 year old adult called Otto.\\n    </user>\\n\\n    <genre>\\n    Comedy\\n    </genre>\\n\\n    <movie>\\n    [{\'title\': \'Big Daddy\', \'genres\': \'Comedy and Drama\', \'plot\': \\"A laz

In [33]:
# Invoke model
response = getPersonalizedEmail(bedrock,model_id, max_tokens_to_sample, prompt_input_json)
print (response)


Subject: Laugh Out Loud with These Comedy Hits on Our Video-On-Demand Platform!

Dear Otto,

As a fellow comedy aficionado, I've handpicked some hilarious movies that are sure to have you in stitches. Get ready to indulge in a side-splitting movie marathon next week on our video-on-demand streaming platform!

First up, we have "Big Daddy" – a laugh riot that follows the misadventures of a lazy law school grad who adopts a kid to impress his girlfriend. But things take an unexpected turn, and he finds himself as an unlikely foster father. This heartwarming comedy will have you chuckling from start to finish as you witness the endearing bond between the two protagonists.

Next, prepare to be swept off your feet by the timeless romantic comedy, "Pretty Woman." When a successful businessman hires a charming prostitute as his escort for social events, they unexpectedly fall in love. This beloved classic is a perfect blend of humor, romance, and heart-melting moments that will leave you wit

### Generating a marketing email for a young adult

In [34]:
print ('User\'s age group')
user_demographic = user_demographic_3
user_demographic

User's age group


'The user is a young adult called Jane.'

In [35]:
print ('User\'s favorite Genre')
user_favorite_genre

User's favorite Genre


'Comedy'

In [36]:
print ('User\'s recommended movies:')

# print each movie in the array
for movie in movie_list:
    print ('Title: '+movie['title'])
    print ('Genres: '+movie['genres'])
    print ('Plot: '+movie['plot'])
    print ()

User's recommended movies:
Title: Big Daddy
Genres: Comedy and Drama
Plot: A lazy law school grad adopts a kid to impress his girlfriend, but everything doesn't go as planned and he becomes the unlikely foster father.

Title: Pretty Woman
Genres: Comedy and Romance
Plot: A man in a legal but hurtful business needs an escort for some social events, and hires a beautiful prostitute he meets... only to fall in love.

Title: 10 Things I Hate About You
Genres: Comedy and Drama and Romance
Plot: A pretty, popular teenager can't go out on a date until her ill-tempered older sister does.



In [37]:
prompt_input_json = generate_personalize_advanced_prompt(user_demographic, user_favorite_genre, movie_list, model_id, max_tokens_to_sample )
prompt_input_json

'{"prompt": "\\n\\nHuman: You are a skilled publicist. Write a high-converting marketing email advertising several movies available in a video-on-demand streaming platform next week, \\n    given the movie and user information below. Your email will leverage the power of storytelling and persuasive language. \\n    You want the email to impress the user, so make it appealing to them based on the information contained in the <user> tags, \\n    and take into account the user\'s favorite genre in the <genre> tags. \\n    The movies to recommend and their information is contained in the <movie> tag. \\n    All movies in the <movie> tag must be recommended. Give a summary of the movies and why the human should watch them. \\n    Put the email between <email> tags.\\n\\n    <user>\\n    The user is a young adult called Jane.\\n    </user>\\n\\n    <genre>\\n    Comedy\\n    </genre>\\n\\n    <movie>\\n    [{\'title\': \'Big Daddy\', \'genres\': \'Comedy and Drama\', \'plot\': \\"A lazy law 

In [38]:
# Invoke model
response = getPersonalizedEmail(bedrock,model_id, max_tokens_to_sample, prompt_input_json)
print (response)


Subject: Laugh Out Loud with Our Must-Watch Comedy Hits!

Hey Jane,

As a fellow comedy enthusiast, I've got an exciting lineup of side-splitting movies for you to enjoy on our video-on-demand platform next week. Get ready to indulge in some seriously hilarious storytelling that will have you in stitches!

First up, we have the classic "Big Daddy" starring Adam Sandler. In this hilarious romp, a lovable slacker inadvertently becomes a foster dad to a young boy, turning his life upside down in the most entertaining way possible. Prepare for gut-busting laughs as Sandler navigates the challenges of unexpected parenthood with his signature brand of slapstick humor.

Next, we've got the beloved romantic comedy "Pretty Woman" starring Julia Roberts and Richard Gere. This charming tale follows a wealthy businessman who hires a charming and spirited escort, only to find himself falling for her wit and charm. Get ready for heartwarming moments, quotable lines, and a feel-good romance that wil

## Wrap up<a class="anchor" id="wrapup"></a>
[Back to top](#top)


In [39]:
# Store variables
%store workshop_dataset_group_arn
%store region
%store role_name

Stored 'workshop_dataset_group_arn' (str)
Stored 'region' (str)
Stored 'role_name' (str)


With that you now have a fully working personalized marketing conten generator.

You'll want to make sure that you clean up all of the resources deployed during this workshop. We have provided a separate notebook which shows you how to identify and delete the resources in [`Media_06_Clean_Up.ipynb`](Media_06_Clean_Up.ipynb).